# Quickstart

In this quickstart you will create a evaluate a LangChain app using ground truth.

### Add API keys
For this quickstart, you will need Open AI keys.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Import from LlamaIndex and TruLens

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

In [ ]:
# Imports main tools:
from trulens_eval import Feedback, Tru, TruChain
from trulens_eval.feedback import GroundTruthAgreement, Huggingface
tru = Tru()

### Create Simple LLM Application

This example uses Langchain with an OpenAI LLM.

In [ ]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide an answer to the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

## Initialize Feedback Function(s)

In [ ]:
golden_set = [
    {"query": "who invented the lightbulb?", "response": "Thomas Edison"},
    {"query": "¿quien invento la bombilla?", "response": "Thomas Edison"}
]


f_groundtruth = Feedback(GroundTruthAgreement(golden_set).agreement_measure).on_input_output()

# Define a language match feedback function using HuggingFace.
hugs = Huggingface()
f_lang_match = Feedback(hugs.language_match).on_input_output()

## Instrument chain for logging with TruLens

In [ ]:
tc = TruChain(chain, feedbacks=[f_groundtruth, f_lang_match])

In [ ]:
# Instrumented query engine can operate like the original:
tc("¿quien invento la bombilla?")
tc("who invented the lightbulb?")

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed